# Pandas Bootcamp

Importanje paketa i učitavanje podataka:

In [1]:
import pandas as pd
import numpy as np

In [2]:
table_1 = pd.read_csv('https://raw.githubusercontent.com/katarinasupe/MS/master/Pandas_Bootcamp.csv')

Pogledajmo kako izgleda dio tablice: 

In [15]:
table_1.head()

,date,num,num2,str,str2
0,2014-02-20,382,880.159693,boy,red
1,2014-02-20,495,545.127367,girl,yellow
2,2014-03-01,920,672.390991,girl,red
3,2014-05-30,646,486.678107,girl,yellow
4,2014-06-04,125,59.971397,boy,green


In [4]:
table_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   date    50 non-null     object 
 1   num     50 non-null     int64  
 2   num2    50 non-null     float64
 3   str     50 non-null     object 
 4   str2    50 non-null     object 
dtypes: float64(1), int64(1), object(3)
memory usage: 2.1+ KB


## Zadaci

* 1. What is the average of column num2?
* 2. Create a new column called "diff". This column returns 1 if column num > column num2 and 0 if it does not. What is the sum of column diff?
* 3. Find the row with the smallest date. Subtract num - num2.
* 4. Group by str and find the sum of column num where the index is boy.
* 5. Find the least popular string in column str2. Filter by str2 where the string equals the least popular string. Find the sum of column num.






## Rješenja

### 1. zadatak

Izračunaj prosjek stupca *num2*.

### Rješenje:

In [5]:
table_1.mean()['num2']

457.4645854799998

### Provjera:

In [6]:
table_1.describe()

,num,num2
count,50.000000,50.000000
mean,565.920000,457.464585
std,281.373515,292.466016
min,0.000000,0.642999
25%,381.250000,175.979159
50%,596.500000,487.904471
75%,794.500000,682.348960
max,988.000000,999.716608


Tablica se po defaultu ispisuje sa 6 decimala. Stoga mi ne vidimo svaki broj s točnom preciznošću. Međutim, kada direktno pristupamo elementu, onda možemo vidjeti kako stvarno izgleda u memoriji. Stoga iz ove tablice idemo provjeriti vrijednost *mean* za *num2*:

In [7]:
table_1.describe().loc['mean', 'num2']

457.4645854799998

Sada vidimo da smo uistinu dobro riješili. 

### 2. zadatak
Kreiraj novi stupac koji se zove "diff". Ovaj stupac vraća $1$ ako je stupac *num* strogo veći od stupca *num2*, $0$ inače. Kolika je suma elemenata stupca *diff*?

Kreiranje novog stupca *diff* : 
`table_1.assign(diff = (np.greater(table_1['num'].to_numpy(),table_1['num2'].to_numpy()))*1)`
(množenje s $1$ tako da `boolean` vrijednosti koje vraća funkcija `greater` iz `numpy` library postanu `int`)

Napomena: `np.greater(v1, v2)` za svaka dva elementa np.arraya vraća `True` ako je v1 veći od v2, `False` inače.

Dohvaćanje stupca *diff* u data frame *table_1* sa `['diff']` te pretvaranje tog stupca u `numpy array` na kojem primijenjujemo funkciju `sum` iz `numpy` libraryja koje će nam dati traženu sumu.

### Rješenje:

In [8]:
np.sum(np.array(table_1.assign(diff = (np.greater(table_1['num'].to_numpy(),table_1['num2'].to_numpy()))*1)['diff']))

26

### Provjera:

In [9]:
diff_column = np.greater(table_1['num'].to_numpy(),table_1['num2'].to_numpy())*1 
np.count_nonzero(diff_column)

26

### 3. zadatak 
Pronađi redak s najmanjim datumom te u njemu oduzmi vrijednosti *num* i *num2*.

### Rješenje (minimalni datum nije jedinstven)

Prilikom testiranja desila mi se situacija da je tablica imala dva jednaka minimalna datuma. Što ako imamo više redaka s istim minimalnim datumom? Možemo te vrijednosti dodati u tablicu, kao novi stupac. Nazovimo ga *subtraction*:

In [10]:
table_1.loc[table_1['date'] == np.amin(np.array(table_1['date']))].assign(subtraction = np.subtract(table_1.loc[table_1['date'] == np.amin(np.array(table_1['date']))]['num'], table_1.loc[table_1['date'] == np.amin(np.array(table_1['date']))]['num2']))

,date,num,num2,str,str2,subtraction
0,2014-02-20,382,880.159693,boy,red,-498.159693
1,2014-02-20,495,545.127367,girl,yellow,-50.127367


Jednostavno možemo ispisati samo tražene vrijednosti. Jedan način je kao DataFrame:

In [11]:
table_1.loc[table_1['date'] == np.amin(np.array(table_1['date']))].assign(subtraction = np.subtract(table_1.loc[table_1['date'] == np.amin(np.array(table_1['date']))]['num'], table_1.loc[table_1['date'] == np.amin(np.array(table_1['date']))]['num2']))['subtraction']

0   -498.159693
1    -50.127367
Name: subtraction, dtype: float64

Drugi način je kao `numpy` array:

In [12]:
np.array(table_1.loc[table_1['date'] == np.amin(np.array(table_1['date']))].assign(subtraction = np.subtract(table_1.loc[table_1['date'] == np.amin(np.array(table_1['date']))]['num'], table_1.loc[table_1['date'] == np.amin(np.array(table_1['date']))]['num2']))['subtraction'])

array([-498.159693,  -50.127367])

### 4. zadatak

Grupiraj po *str* i izračunaj sumu stupca *num* gdje je index *boy*.

`table_1.groupby('str').sum()` će zbrojiti elemente svih stupaca i grupirati ih po retcima *boy* i *girl*. Mi želimo samo sumu stupca *num* gdje je index *boy*, stoga koristeći se `loc` pristupamo baš tom elementu tablice, a pomoću `item()` iz `numpy` paketa vraćamo standardni Python skalar.  

In [13]:
np.array(table_1.groupby('str').sum().loc[['boy'], ['num']]).item()

14793

### 5. zadatak

Pronađi najmanje popularan string u stupcu *str2* (onaj koji se najmanje puta pojavljuje). Filtriraj tablicu tako da ostanu podatci kada je *str2* jednak najmanje popularnom stringu. Nađi sumu stupca *num*. 

### Rješenje

Funkcija `value_counts` će nam prebrojati ponavljanja svih jedinstvenih vrijednosti u odabranom stupcu *str2*, a onda će funkcija `idxmin` vratiti vrijednost koja se najmanje puta pojavljuje. Zatim pomoću `loc` filtriramo početnu tablicu tako da ostanu podatci kada je *str2* jednak najmanje popularnom stringu. Za kraj iskoristimo funkciju `sum` te odaberemo stupac *num* da dobijemo traženu vrijednost.  

In [14]:
table_1.loc[table_1['str2'] == table_1['str2'].value_counts().idxmin()].sum()['num']

3831